In [1]:
from __future__ import print_function

# import glob
import os

import cv2
import numpy as np
import tensorflow as tf

In [18]:
class opt:
    modelname = 'casapose_c_gcu5'
    estimate_confidence = 1
    estimate_coords = 1
    confidence_regularization = 1
    object = 'obj_000001,obj_000005,obj_000006,obj_000008,obj_000009,obj_000010,obj_000011,obj_000016'

    no_points = 9
    save_debug_batch = 0

    imagesize = (448, 448)
    imagesize_test = (480, 640)
    crop_factor = 0.933333333
    pretrained = 1
    manualseed = 1237

    # losses
    mask_loss_weight = 1.0
    vertex_loss_weight = 0.5
    proxy_loss_weight = 0.015
    keypoint_loss_weight = 0.007
    filter_vertex_with_segmentation = 1
    filter_high_proxy_errors = 0
    use_bpnp_reprojection_loss = 0
    max_keypoint_pixel_error = 12.5

    # learning rate
    lr = 0.001
    lr_decay = 0.5
    lr_epochs_steps = 50,75,90

    # general
    gpuids = 0,1
    loginterval = 10
    epochs = 100
    batchsize = 4
    saveinterval = 5
    validationinterval = 1

    # data preprocessing
    workers = 0
    prefetch = 10

    # augmentation
    translation = 0
    rotation = 0
    noise = 0.0001
    brightness = 0.001
    contrast = 0.001
    saturation = 0.001
    hue = 0.001
    use_imgaug = 1

    # test
    min_object_size_test = 200
    write_poses = 0
    save_eval_batches = 0

    # output
    net = 'training_checkpoints'
    outf = 'train_casapose_8_16_objects'

    # config
    train_vectors_with_ground_truth = 1
    load_h5_weights = 0
    copy_weights_from_backup_network = 0
    copy_weights_add_confidence_maps = 0
    objects_in_input_network = 8
    objects_to_copy = 1
    objects_to_copy_list = 'config/objects_to_copy.csv'
    
    confidence_filter_estimates = 1
    confidence_choose_second = 0
    train_vectors_with_ground_truth = 0
    datatest_wxyz_quaterion = 0
    filter_test_with_gt = 0

In [3]:
os.environ["CASAPOSE_INFERENCE"] = "True"

from casapose.data_handler.vectorfield_dataset import VectorfieldDataset
from casapose.data_handler.image_only_dataset import ImageOnlyDataset
from casapose.pose_estimation.pose_evaluation import (
    estimate_and_evaluate_poses,
    evaluate_pose_estimates,
)
from casapose.pose_estimation.voting_layers_2d import CoordLSVotingWeighted
from casapose.pose_models.tfkeras import Classifiers
from casapose.utils.config_parser import parse_config
#from casapose.utils.dataset_utils import save_eval_batch  # , save_eval_comparison
from casapose.utils.inf_dataset_utils import save_eval_batch  # , save_eval_comparison
from casapose.utils.image_utils import get_all_vectorfields
# from casapose.utils.io_utils import write_poses
from casapose.utils.loss_functions import (
    keypoint_reprojection_loss,
    proxy_voting_dist,
    proxy_voting_loss_v2,
    smooth_l1_loss,
)

/usr/local/lib/python3.8/dist-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)
/usr/local/lib/python3.8/dist-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `FrequencyNoiseAlpha()` is deprecated. Use `BlendAlphaFrequencyNoise` instead. FrequencyNoiseAlpha is deprecated. Use BlendAlphaFrequencyNoise instead. The order of parameters is the same. Parameter 'first' was renamed to 'foreground'. Parameter 'second' was renamed to 'background'.
  warn_deprecated(msg, stacklevel=3)


In [4]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

2023-03-31 11:06:21.877963: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 11:06:21.967468: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 11:06:22.003550: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 11:06:22.005242: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

In [5]:
def compute_loss(
    output_seg,
    target_seg,
    output_vertex,
    target_vertex,
    target_points,
    mask_loss_weight=1.0,
    vertex_loss_weight=1.0,
    proxy_loss_weight=1.0,
    kp_loss_weight=0.0,
    kp_loss=None,
):

    oc = np.int32(target_seg.shape[3] - 1)  # object count
    vc = target_points.shape[3] * 2  # vertex count
    kp_loss = tf.constant(0.0, dtype=tf.float32)
    mask_loss = tf.reduce_mean(
        input_tensor=tf.nn.softmax_cross_entropy_with_logits(labels=target_seg, logits=output_seg)
    )
    separated_vectors = oc > 1 and output_vertex.shape[-1] == (oc * vc)  # original pvnet with multiple objects

    if kp_loss is None:
        kp_loss = tf.constant(0.0, dtype=tf.float32)

    if separated_vectors:  #
        vertex_loss = sum(
            smooth_l1_loss(
                output_vertex[:, :, :, i * vc : (i + 1) * vc],
                target_vertex[:, :, :, i * vc : (i + 1) * vc],
                target_seg[:, :, :, i + 1 : i + 2],
            )
            for i in range(oc)
        )
        proxy_loss = sum(
            proxy_voting_loss_v2(
                output_vertex[:, :, :, i * vc : (i + 1) * vc],
                target_points[:, i : i + 1, :, :, :],
                vertex_one_hot_weights=target_seg[:, :, :, i + 1 : i + 2],
                vertex_weights=target_seg[:, :, :, i + 1 : i + 2],
            )
            for i in range(oc)
        )
    else:
        vertex_loss = smooth_l1_loss(
            output_vertex,
            target_vertex,
            target_seg[:, :, :, 0:1],
            invert_weights=True,
        )
        proxy_loss = proxy_voting_loss_v2(
            output_vertex,
            target_points,
            vertex_one_hot_weights=target_seg[:, :, :, 1:],
            vertex_weights=target_seg[:, :, :, 0:1],
            invert_weights=True,
            loss_per_object=False,
        )

    loss = (
        tf.multiply(mask_loss, mask_loss_weight)
        + tf.multiply(proxy_loss, proxy_loss_weight)
        + tf.multiply(vertex_loss, vertex_loss_weight)
        + tf.multiply(kp_loss, kp_loss_weight)
    )
    return [loss, mask_loss, vertex_loss, proxy_loss, kp_loss]

In [6]:
#opt = parse_config()

In [7]:
# if not os.path.exists(opt.evalf):
#     os.makedirs(opt.evalf)

# checkpoint_path = opt.outf + "/" + opt.net

# frozen_path = opt.outf + "/frozen_model"
# img_out_path = opt.outf + "/control_output"


# def create_dir(path):
#     try:
#         os.makedirs(path)
#     except OSError:
#         pass


# create_dir(img_out_path)
# create_dir(frozen_path)

# save the hyper parameters passed
# with open(opt.evalf + "/header_eval.txt", "w") as file:
#     file.write(str(opt))

# set the manual seed.
# np.random.seed(opt.manualseed)
# tf.random.set_seed(opt.manualseed)

test_dataset = None

device_ids = []
if len(opt.gpuids) == 1 and opt.gpuids[0] < 0:
    device_ids.append("/cpu:0")
else:
    device_ids.append("/gpu:{}".format(opt.gpuids[0]))
print(device_ids)

objectsofinterest = [x.strip() for x in opt.object.split(",")]
no_objects = len(objectsofinterest)
separated_vectorfields = opt.modelname == "pvnet"

testingdata = None
normal_imgs = [0.5, 0.5]

use_split = False
# if opt.data == opt.datatest:
#     print("split datasets with ratio {}".format(opt.train_validation_split))
#     use_split = True

test_batches = 0


#print("testing data: {} batches".format(test_batches))

# if opt.backbonename != "resnet18":
#     print(opt.backbonename + " is not a supported backbone.")
#     exit()


input_segmentation_shape = None
# if opt.train_vectors_with_ground_truth is True:
#     input_segmentation_shape = (
#         opt.imagesize_test[0],
#         opt.imagesize_test[1],
#         1 + no_objects,
#     )


height = 480 #Taken from config_8.ini
width = 640 # Taken from config_8.ini

CASAPose = Classifiers.get(opt.modelname)
ver_dim = 9 * 2 # Taken from config_8.ini
if opt.modelname == "pvnet":
    ver_dim = ver_dim * no_objects

if opt.estimate_confidence:
    assert separated_vectorfields is not None, "confidence not compatitble with this model"
    ver_dim += 9 # Taken from config_8.ini

net = CASAPose(
    ver_dim=ver_dim,
    seg_dim=1 + no_objects,
    input_shape=(height, width, 3),
    input_segmentation_shape=input_segmentation_shape,
    weights="imagenet",
    base_model="resnet18", # Hardcoded param
)

2023-03-31 11:06:22.405416: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 11:06:22.406288: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 11:06:22.407013: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 11:06:22.407905: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 11:06:22.408611: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from S

['/gpu:0']
[<KerasTensor: shape=(None, 240, 320, 64) dtype=float32 (created by layer 'relu0')>, <KerasTensor: shape=(None, 120, 160, 64) dtype=float32 (created by layer 'stage2_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 128) dtype=float32 (created by layer 'stage3_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 256) dtype=float32 (created by layer 'stage4_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 512) dtype=float32 (created by layer 'relu1')>]


In [8]:
# checkpoint_prefix = os.path.join(checkpoint_path, "ckpt")
# checkpoint = tf.train.Checkpoint(network=net)  # , optimizer=optimizer)


net_path = "/workspace/CASAPose/data/pretrained_models/result_w.h5" # CHANGE TO .H5 FILEPATH
net.load_weights(net_path, by_name=True, skip_mismatch=True)


# elif opt.net != "":
#     checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path)).expect_partial()


In [9]:
for layer in net.layers:
    layer.trainable = False

net.summary()

# if os.path.exists(opt.evalf + "/poses_out/"):
#     files = sorted(glob.glob(opt.evalf + "/poses_out/*/" + "*.txt"))
#     for f in files:
#         os.remove(f)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 480, 640, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             [(None, 240, 320, 6  11186889    ['data[0][0]']                   
                                4),                                                               
                                 (None, 120, 160, 6                                               
                                4),                                                               
                                 (None, 60, 80, 128                                         

In [10]:
def runnetwork(loader_iterator, batches):
    # @tf.function
    # def tf_test_step(img_batch):
    #    return test_step(img_batch)

    # @tf.function
    # def test_step(img_batch):
    for batch_idx in range(batches):

        img_batch = loader_iterator.get_next()

        loss = 0

        target_seg = img_batch[1]
        target_vertex = img_batch[3]
        keypoints = img_batch[4]
        cam_mat = img_batch[5]
        diameters = img_batch[6]
        offsets = img_batch[7]
        filtered_seg = img_batch[8]
        poses_gt = img_batch[10]
        confidence = None
        kp_loss = None
        no_features = target_vertex.shape[3]  # vertex count
        no_points = opt.no_points
        no_objects = target_seg.shape[3]

        separated_vectorfields = opt.modelname == "pvnet"

        target_dirs = get_all_vectorfields(
            target_seg,
            target_vertex,
            filtered_seg,
            separated_vectorfields,
        )

        net_input = [img_batch[0]]
        # if opt.train_vectors_with_ground_truth:
        #     net_input.append(target_seg)

        # start_time = tf.timestamp()
        output_net = net(net_input, training=False)  # all stages are present here

        if opt.estimate_confidence:
            output_seg, output_dirs, confidence = tf.split(output_net, [no_objects, no_points * 2, -1], 3)
        else:
            output_seg, output_dirs = tf.split(output_net, [no_objects, -1], 3)

        if opt.estimate_coords:
            if opt.train_vectors_with_ground_truth:
                coordLSV_in = [target_seg, output_dirs, confidence]
            else:
                coordLSV_in = [output_seg, output_dirs, confidence]

            coords = CoordLSVotingWeighted(
                name="coords_ls_voting",
                num_classes=no_objects,
                num_points=no_points,
                filter_estimates=opt.confidence_filter_estimates,
                output_second_largest_component=opt.confidence_choose_second,
            )(coordLSV_in)

            kp_loss, poses_est, points_est = keypoint_reprojection_loss(
                coords,
                output_seg,
                poses_gt,
                keypoints,
                target_seg,
                cam_mat,
                offsets,
                confidence,
                min_num=opt.min_object_size_test,
                min_num_gt=1,
                use_bpnp_reprojection_loss=opt.use_bpnp_reprojection_loss,
                estimate_poses=True,
                filter_with_gt=opt.filter_test_with_gt,
            )

        if opt.estimate_coords:
            pose_stats, estimated_poses, estimated_points = evaluate_pose_estimates(
                points_est,
                poses_est,
                poses_gt,
                target_seg,
                keypoints,
                cam_mat,
                diameters,
                evaluation_points=mesh_vertex_array,
                object_points_3d_count=mesh_vertex_count,
                min_num=1,
            )
            estimated_poses = tf.squeeze(estimated_poses, axis=2)
        else:
            pose_stats, estimated_poses, estimated_points = estimate_and_evaluate_poses(
                output_seg,
                target_seg,
                output_dirs,
                poses_gt,
                keypoints,
                cam_mat,
                diameters,
                offsets,
                evaluation_points=mesh_vertex_array,
                object_points_3d_count=mesh_vertex_count,
                min_num=1,
            )

        # end_time = tf.timestamp()
        # time_needed = end_time - start_time

        loss = compute_loss(
            output_seg,
            target_seg,
            output_dirs,
            target_dirs,
            target_vertex,
            opt.mask_loss_weight,
            opt.vertex_loss_weight,
            opt.proxy_loss_weight,
            opt.keypoint_loss_weight,
            kp_loss=kp_loss,
        )
        loss.append(pose_stats)

        _, object_loss_values = proxy_voting_dist(
            output_dirs,
            target_vertex,
            vertex_one_hot_weights=target_seg[:, :, :, 1:],
            vertex_weights=target_seg[:, :, :, 0:1],
            invert_weights=True,
        )
        loss.append(object_loss_values)
        # loss.append(time_needed)

        # if opt.write_poses:
        #     write_poses(
        #         tf.squeeze(poses_gt, 0),
        #         tf.squeeze(estimated_poses, 0),
        #         objectsofinterest,
        #         img_batch[12],
        #         opt.evalf + "/poses_out/",
        #     )

        # if opt.save_eval_batches:

        beta = tf.cast(1e6, dtype=output_seg.dtype)
        hot_seg = tf.expand_dims(tf.expand_dims(tf.nn.softmax(output_seg * beta), -1), -1)[:, :, :, 1:, :, :]
        w = tf.math.softplus(confidence)
        hot_seg_ = tf.squeeze(hot_seg, axis=-1)
        w_ = hot_seg_ * tf.expand_dims(w, axis=-2)
        w_max = tf.reduce_max(w_, axis=[0, 1, 2], keepdims=True)
        w_min = tf.reduce_min(w_, axis=[0, 1, 2], keepdims=True)
        w_ = tf.math.divide_no_nan(tf.subtract(w_, w_min), tf.subtract(w_max, w_min))
        confidence = tf.reduce_sum(w_, axis=-2)

        add_correct = loss[5][1]

        image = save_eval_batch(
                img_batch,
                output_seg,
                target_dirs,
                output_dirs,
                estimated_poses,
                estimated_points,
                no_objects - 1,
                no_features,
                # path_out=opt.evalf + "/visual_batch_eval_mask",
                confidence=confidence,
                add_correct=add_correct,
                )
            
        tf.print("Estimated Coordinates: {}".format(estimated_points), summarize=-1)
        tf.print("Estimated Poses: {}".format(estimated_poses), summarize=-1)
        tf.print("Segmentation Masks: {}".format(output_seg), summarize=-1)
        tf.print("Image: {}".format(type(image)), summarize=-1)
        #tf.print("Labels: TESTING") #{}".format(test_pose_count_gt), summarize=-1)
        tf.print("Objects of Interest: {}".format(objectsofinterest), summarize=-1)
        tf.print("length: {}".format(len(objectsofinterest)), summarize=-1)
        tf.print("Number of Objects: {}".format(no_objects), summarize=-1)
        

        return image
        # return loss, estimated_points, estimated_poses, output_seg
        # return estimated_points, estimated_poses, output_seg

        # def test_pose_step(dataset_inputs):
            # if opt.save_eval_batches or opt.write_poses:
                # loss, 
            # estimated_points, estimated_poses, output_seg = test_step(dataset_inputs)
            # else:
                # loss, estimated_points, estimated_poses, output_seg = tf_test_step(dataset_inputs)

            # return [
            #     loss[0],
            #     loss[1],
            #     loss[2],
            #     loss[3],
            #     loss[4],
            #     loss[5][0],
            #     loss[5][1],
            #     loss[5][2],
            #     loss[5][3],
            #     loss[5][4],
            #     loss[5][5],
            #     loss[5][6],
            #     loss[5][7],
            #     loss[6],
            #     loss[7],
            # ], return estimated_points, estimated_poses, output_seg

    # test_loss = tf.zeros([5], dtype=tf.float32)
    # test_pose_2d_count = tf.zeros([no_objects], dtype=tf.float32)
    # test_pose_3d_count = tf.zeros([no_objects], dtype=tf.float32)
    # test_pose_count_gt = tf.zeros([no_objects], dtype=tf.float32)
    # test_pose_count_fp = tf.zeros([no_objects], dtype=tf.float32)
    # test_pose_err_2d = tf.zeros([no_objects], dtype=tf.float32)
    # test_pose_err_3d = tf.zeros([no_objects], dtype=tf.float32)
    # missed_object_count = tf.zeros([no_objects], dtype=tf.float32)

    # for batch_idx in range(batches):

        # img_batch = loader_iterator.get_next()
        # loss, 
        # image, estimated_points, estimated_poses, output_seg = test_step(img_batch)
        # estimated_points, estimated_poses, output_seg = test_pose_step(img_batch)
        # test_pose_2d_count += loss[5]
        # test_pose_3d_count += loss[6]
        # test_pose_count_gt += loss[7]
        # test_pose_count_fp += loss[12]
        # test_pose_err_2d += loss[9]
        # test_pose_err_3d += loss[10]
        # missed_object_count += loss[11]
        # test_loss += loss[0:5]
        
        # tf.print("Estimated Coordinates: {}".format(estimated_points), summarize=-1)
        # tf.print("Estimated Poses: {}".format(estimated_poses), summarize=-1)
        # tf.print("Segmentation Masks: {}".format(output_seg), summarize=-1)
        # tf.print("Image: {}".format(type(image)), summarize=-1)
        # tf.print("Labels: TESTING") #{}".format(test_pose_count_gt), summarize=-1)

    # test_loss /= batches

    # err_2d = tf.math.divide_no_nan(test_pose_2d_count, test_pose_count_gt)
    # err_3d = tf.math.divide_no_nan(test_pose_3d_count, test_pose_count_gt)
    # detection_count = test_pose_count_gt - missed_object_count + test_pose_count_fp
    # detection_count = tf.where(test_pose_count_gt == 0.0, 0.0, detection_count)
    # precision = tf.math.divide_no_nan(test_pose_3d_count, detection_count)

    # return loader_iterator, image, estimated_points, estimated_poses, output_seg


# cap = cv2.VideoCapture('drive/MyDrive/MIDS/w251/final_project/desk.mp4')

#img = cv2.imread("color.png")
#test_dataset = ImageOnlyDataset(img)

In [11]:
test_dataset = VectorfieldDataset(
    root="/workspace/CASAPose/import_data/test/test", # Hardcoded param
    path_meshes="/workspace/CASAPose/import_data/test/models", # Hardcoded param
    path_filter_root=None, # Hardcoded param
    color_input=True, # Hardcoded param
    no_points=9, # Taken from config_8.ini
    objectsofinterest=objectsofinterest,
    noise=0.00001,
    data_size=None,
    save=False, # Hardcoded param
    normal=normal_imgs,
    contrast=0.00001,
    brightness=0.00001,
    hue=0.00001,
    saturation=0.00001,
    random_translation=(0, 0),
    random_rotation=0,
    random_crop=False,
    use_validation_split=use_split,
    use_train_split=False,
    train_validation_split=None,
    output_folder=None,
    visibility_filter=False,
    separated_vectorfields=(opt.modelname == "pvnet"),
    wxyz_quaterion_input=opt.datatest_wxyz_quaterion,
)

/workspace/CASAPose/import_data/test/models/obj_000001/obj_000001.ply
/workspace/CASAPose/import_data/test/models/obj_000002/obj_000002.ply
/workspace/CASAPose/import_data/test/models/obj_000003/obj_000003.ply
/workspace/CASAPose/import_data/test/models/obj_000004/obj_000004.ply
/workspace/CASAPose/import_data/test/models/obj_000005/obj_000005.ply
/workspace/CASAPose/import_data/test/models/obj_000006/obj_000006.ply
/workspace/CASAPose/import_data/test/models/obj_000007/obj_000007.ply
/workspace/CASAPose/import_data/test/models/obj_000008/obj_000008.ply
/workspace/CASAPose/import_data/test/models/obj_000009/obj_000009.ply
/workspace/CASAPose/import_data/test/models/obj_000010/obj_000010.ply
/workspace/CASAPose/import_data/test/models/obj_000011/obj_000011.ply
/workspace/CASAPose/import_data/test/models/obj_000012/obj_000012.ply
/workspace/CASAPose/import_data/test/models/obj_000013/obj_000013.ply
/workspace/CASAPose/import_data/test/models/obj_000014/obj_000014.ply
/workspace/CASAPose/

In [12]:
print("Dataset length: ", len(test_dataset))
testingdata, test_batches = test_dataset.generate_dataset(
    1, 1, 0, (480, 640), 1.0, 1, no_objects, shuffle=True
)

Dataset length:  5
Instructions for updating:
Use fn_output_signature instead


In [13]:
mesh_vertex_array, mesh_vertex_count = test_dataset.generate_object_vertex_array()

In [14]:
testingdata_iterator = iter(testingdata)

In [15]:
testingdata

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 480, 640, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 480, 640, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 480, 640, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 8, 1, 9, 2), dtype=tf.float32, name=None), TensorSpec(shape=(None, 8, 1, 9, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 8, 1, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None), TensorSpec(shape=(None, 480, 640, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 8, 1, 8, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 8, 1, 3, 4), dtype=tf.float32, name=None), TensorSpec(shape=(None, 8, 1, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.string, name=None))>

In [16]:
testingdata_iterator.get_next()[1]

<tf.Tensor: shape=(1, 480, 640, 9), dtype=float32, numpy=
array([[[[1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.]],

        [[1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.]],

        [[1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ...,

In [19]:
img = runnetwork(testingdata_iterator, int(test_batches))

2023-03-31 11:08:28.723112: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator Assert/Assert
2023-03-31 11:08:28.732176: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator Assert_1/Assert
2023-03-31 11:08:28.732275: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator Assert_2/Assert
2023-03-31 11:08:28.732324: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator Assert_3/Assert


[TensorShape([8, 9, 2]), TensorShape([8, 9, 3]), TensorShape([3, 3])]
[0 0 0 0 0 0 0 0]
Estimated Coordinates: [[[[374.26462 337.24667]
   [352.54822 336.1311 ]
   [375.24445 348.34293]
   [361.89462 351.3888 ]
   [386.0225  329.66934]
   [368.78754 331.5153 ]
   [370.8289  349.78806]
   [362.72827 326.75467]
   [381.85168 337.52258]]

  [[318.45404 409.8459 ]
   [294.55573 406.6304 ]
   [342.70773 381.16058]
   [294.7497  445.60455]
   [315.04025 372.78464]
   [317.46225 453.77698]
   [283.0346  410.58612]
   [323.21164 425.7287 ]
   [352.213   398.33746]]

  [[  0.        0.     ]
   [  0.        0.     ]
   [  0.        0.     ]
   [  0.        0.     ]
   [  0.        0.     ]
   [  0.        0.     ]
   [  0.        0.     ]
   [  0.        0.     ]
   [  0.        0.     ]]

  [[366.07794 283.95816]
   [362.4618  227.53452]
   [407.22583 305.69046]
   [355.90067 323.29968]
   [341.05026 289.85947]
   [388.44357 256.68237]
   [396.15848 282.5333 ]
   [335.8342  305.90555]
   [364.

In [20]:
print(type(img))

<class 'numpy.ndarray'>


In [21]:
cv2.imwrite("final_image.png", img)

True